In [1]:
pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 3.9 MB/s eta 0:00:00


In [ ]:
import os
import re
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from collections import defaultdict
from sentence_transformers import SentenceTransformer
import spacy
from sklearn.metrics.pairwise import cosine_similarity
from concurrent.futures import ThreadPoolExecutor, as_completed
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#Load the embeddings model and the NLP model
model = SentenceTransformer('paraphrase-mpnet-base-v2')  # Pre-trained model of embeddings
nlp = spacy.load('en_core_web_sm')  # Model NLP

# Customized list of irrelevant words in company names
EXTRA_STOP_WORDS = {"inc", "corp", "co", "ltd", "group", "corporation", "limited", "company", "mall", "road", "street"}

# Function to clean up company names using spaCy and eliminate stop words
def clean_firm_name(name):
    doc = nlp(name.lower())
    cleaned_name = ' '.join(
        [token.lemma_ for token in doc if not token.is_stop and token.is_alpha and token.lemma_ not in EXTRA_STOP_WORDS]
    )
    return cleaned_name

# Function to calculate embeddings of company names in batches
def get_embeddings(names):
    batch_size = 90  # Sets this value according to the available memory
    embeddings = []
    for i in range(0, len(names), batch_size):
        batch = names[i:i + batch_size]
        embeddings.extend(model.encode(batch, convert_to_tensor=True))
    return np.array(embeddings)


# Function for grouping similar names using DBSCAN
def group_similar_names_with_dbscan(embeddings, names, eps=0.3, min_samples=3):
    clustering = DBSCAN(eps=eps, min_samples=min_samples, metric='cosine').fit(embeddings)
    grouped_names = defaultdict(list)

    for idx, label in enumerate(clustering.labels_):
        grouped_names[label].append(names[idx])

    # Filtrar outliers (label -1 indica ruido)
    grouped_names = {k: v for k, v in grouped_names.items() if k != -1}
    return grouped_names

# Function for filtering names using cosine similarity after DBSCAN
def filter_similar_names(grouped_names, embeddings, names, threshold=0.8):
    refined_groups = defaultdict(list)

    for label, group_names in grouped_names.items():
        group_indices = [names.index(name) for name in group_names]
        name_vectors = [embeddings[idx] for idx in group_indices]
        similarity_matrix = cosine_similarity(name_vectors)

        for i, name1 in enumerate(group_names):
            for j, name2 in enumerate(group_names):
                if i != j and similarity_matrix[i][j] >= threshold:
                    refined_groups[label].append(name1)
                    refined_groups[label].append(name2)

        # Eliminate duplicates within each group and keep only names with sufficient similarity
        refined_groups[label] = list(set(refined_groups[label]))

    # Filter out groups that were left empty
    refined_groups = {k: v for k, v in refined_groups.items() if len(v) > 1}
    return refined_groups

# Function to process names in parallel and calculate embeddings
def parallel_process_firm_names(group, text_column, max_workers=None):
    if max_workers is None:
        max_workers = os.cpu_count() * 2

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        tqdm.pandas(desc="Cleaning Firm Names")
        cleaned_names = list(executor.map(clean_firm_name, group[text_column].tolist()))

    # Get embeddings of the cleaned names in batches
    embeddings = get_embeddings(cleaned_names)

    # Grouping similar names using DBSCAN
    grouped_names = group_similar_names_with_dbscan(embeddings, cleaned_names, eps=0.7, min_samples=2)

    # Filtering names using cosine similarity to improve the quality of the groups
    refined_grouped_names = filter_similar_names(grouped_names, embeddings, cleaned_names, threshold=0.7)

    return refined_grouped_names, pd.DataFrame({text_column: cleaned_names, 'cleaned_name': cleaned_names})

# Function for assigning unique IDs and choosing the best name
def assign_unique_ids_and_best_names(grouped_names, original_group, text_column, country_name):
    cleaned_data = []

    for group, names in grouped_names.items():
        best_name = max(names, key=len)  # Choosing the longest name as the best name
        original_names = original_group[original_group['cleaned_name'].isin(names)][text_column].tolist()

        cleaned_data.append({
            'cleaned_ID': id(best_name),
            'cleaned_name': best_name,
            'original_firm_names': original_names,
            'country': country_name
        })

    return cleaned_data



# Main function to clean, group and assign IDs by group with parallelization by country using ThreadPoolExecutor
def clean_and_group_by_country(df, text_column, country_column, max_workers=None):
    all_cleaned_data = []

    # Group by country
    grouped_df = df.groupby(country_column)

    # Create a progress bar with the total number of countries to process
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {
            executor.submit(
                parallel_process_firm_names, group, text_column, max_workers
            ): (country, group) for country, group in grouped_df
        }

        # Configure tqdm manually
        with tqdm(total=len(futures), desc="Processing Countries") as pbar:
            for future in as_completed(futures):
                country, group = futures[future]
                try:
                    grouped_names, original_group = future.result()
                    cleaned_data = assign_unique_ids_and_best_names(grouped_names, original_group, text_column, country)
                    all_cleaned_data.extend(cleaned_data)
                except Exception as e:
                    print(f"Error processing country {country}: {e}")

                # Update the progress bar
                pbar.update(1)

    # Create a new DataFrame with the results.
    cleaned_df = pd.DataFrame(all_cleaned_data)

    return cleaned_df

# Load data up to 2020 and new data for 2021
df_old = pd.read_csv('ForeignNames_2019_2020.csv')
df_new = pd.read_csv('ForeignNames_2021.csv')
# Add a 'new' column to identify the data for 2021
df_old['new'] = 0
df_new['new'] = 1
# Join both DataFramess
df = pd.concat([df_old, df_new], ignore_index=True)
df['cleaned_ID'] = df['foreign']
df['cleaned_ID'] = df['cleaned_ID'].str.lower()
df['foreign'] = df['foreign'].str.lower()
df = df.dropna()
df= df.sample(n=60000)
df= df.groupby(['cleaned_ID','foreign', 'foreigncountry_cleaned', 'shpmtyear']).count().reset_index()
processed_df = clean_and_group_by_country(df,'foreign', 'foreigncountry_cleaned', max_workers=40)
# Merge again with the original data to obtain the original company name and the column 'new'
processed_df = processed_df.merge(df[['foreign', 'cleaned_ID', 'new']], on='cleaned_ID', how='left')
# Get the 'old_name' where 'new' is 0, i.e. from 2019-2020 data
processed_df['old_name'] = processed_df.apply(lambda row: row['foreign'] if row['new'] == 0 else None, axis=1)
# Rename column 'foreign' to 'cleaned_name' for clarity
processed_df.rename(columns={'foreign': 'cleaned_name'}, inplace=True)
# Explode
df_explotado = processed_df.explode('original_firm_names')
# Upload Country_Name_ISO3.csv
iso_df = pd.read_csv('Country_Name_ISO3.csv')
# Ensure that the columns have the same data type.
df_explotado['country'] = df_explotado['country'].astype(str)
iso_df['country_name'] = iso_df['country_name'].astype(str)
# Perform the merge between the two databases using the corresponding columns
df_1 = df_explotado.merge(iso_df[['country_name', 'country_iso3']],
                     how='left',
                     left_on='country',
                     right_on='country_name')
# Export the resulting DataFrame to a CSV file with the specified name
df_changed.to_csv('outputfile_Maria_2.csv', index=False)

Processing Countries:  90%|████████▉ | 193/215 [17:58<10:50, 29.57s/it]